## Webscrape kulturpersoner.uppsalakyrkogardar
version 0.2

checks uppsalakyrkogardar and compare with Wikidata. 

* This [notebook](https://github.com/salgo60/open-data-examples/blob/master/Check%20WD%20kulturpersoner.uppsalakyrkogardar.ipynb)
* Wikidata SPARQL https://w.wiki/PgP
* Webpage to check https://kulturpersoner.uppsalakyrkogardar.se/galleri/

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2020-09-29 13:51:05.533821


In [2]:
#https://kulturpersoner.uppsalakyrkogardar.se/galleri/
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
def cleanupstring(s):
    out_s = s
    while '  ' in out_s:
        out_s = out_s.strip().replace('  ', ' ')
    return out_s, len(s)-len(out_s)

urls = [
    'https://kulturpersoner.uppsalakyrkogardar.se/galleri/']

newList = []

for url in urls:

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    soup.findAll('a')
    one_a_tag = soup.findAll('a')[0]
    #    print ( one_a_tag )
    #   link = one_a_tag['href']
    time.sleep(1)
    for i in range(1, len(soup.findAll('a'))):
        new_item = dict()
        one_a_tag = soup.findAll('a')[i]
        one_a_tag_href = one_a_tag['href']
        #print(one_a_tag_href)
        #print("\t",one_a_tag.text)
        if   "karta" not in one_a_tag_href \
                and not "Kyrkogårdsvandring" in one_a_tag.text \
                and not "Search" in one_a_tag.text \
                and not "Start" in one_a_tag.text \
                and not "Vandringar" in one_a_tag.text \
                and not "Guidad" in one_a_tag.text \
                and not "galleri" in one_a_tag_href \
                and not "om-oss"  in one_a_tag_href \
                and not "kyrkogardsexpedition"  in one_a_tag_href:
            #print(one_a_tag.text,"|",one_a_tag.text.replace(" ","_").replace("%C3%84","Ä"),"|",one_a_tag_href.replace("/genealogi/",""))
            
            #person = one_a_tag.text.replace("https://kulturpersoner.uppsalakyrkogardar.se/","").replace("-"," ").replace("%C3%84","Ä")
            #person = one_a_tag.text.replace("https://kulturpersoner.uppsalakyrkogardar.se/","")
            new_item["kulturgravname"] = one_a_tag.text
            new_item["href"] = one_a_tag['href'].replace("https://kulturpersoner.uppsalakyrkogardar.se/","").replace("/","")
            #print(new_item)
            newList.append(new_item)
print (len(newList) ," antal poster")

128  antal poster


In [3]:
import pandas as pd  
Kulturgravar = pd.DataFrame(newList)

In [4]:
Kulturgravar

,kulturgravname,href
0,Alfred Dahlgren,alfred-dahlgren
1,Rosalie Olivecrona,rosalie-olivecrona
2,Hans Rosling,hans-rosling
3,Emma Schenson,emma-schenson
4,Sonja Lyttkens,sonja-lyttkens
...,...,...
123,Uno von Troil,uno-von-troil
124,Viktor Persson,viktor-persson
125,Vivi Täckholm,vivi-tackholm
126,Yngve Svalander,yngve-svalander


Wikidata 

In [5]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# https://w.wiki/crt

query = """select ?person (REPLACE(STR(?person),".*Q","Q") AS ?qid) ?personLabel ?refURL ?href
{
   hint:Query hint:optimizer "None" .
   ?person wdt:P119 wd:Q4353116;
           p:P119 ?burial_statement. #Place of burial
   {?burial_statement prov:wasDerivedFrom ?ref}
   {?ref pr:P854 ?refURL} 
   FILTER (CONTAINS(str(?refURL),'kulturpersoner')) .  
   BIND(REPLACE(REPLACE(str(?refURL), "https://kulturpersoner.uppsalakyrkogardar.se/", ""),"/","") AS ?href)
 
   SERVICE wikibase:label { bd:serviceParam wikibase:language "sv"}
}
Order by ?personLabel
"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDKulturGravar = get_sparql_dataframe(endpoint_url, query)


In [6]:
WDKulturGravar

,person,qid,personLabel,refURL,href
0,http://www.wikidata.org/entity/Q63992891,Q63992891,Adolf Fredrik Nyman,https://kulturpersoner.uppsalakyrkogardar.se/a...,adolf-nyman
1,http://www.wikidata.org/entity/Q2913935,Q2913935,Adolf Murray,https://kulturpersoner.uppsalakyrkogardar.se/a...,adolph-murray
2,http://www.wikidata.org/entity/Q2653181,Q2653181,Adolf Noreen,https://kulturpersoner.uppsalakyrkogardar.se/a...,adolf-noreen
3,http://www.wikidata.org/entity/Q364399,Q364399,Adolph Törneros,https://kulturpersoner.uppsalakyrkogardar.se/a...,adolph-torneros
4,http://www.wikidata.org/entity/Q4949712,Q4949712,Agnes Geijer,https://kulturpersoner.uppsalakyrkogardar.se/a...,agnes-geijer
...,...,...,...,...,...
116,http://www.wikidata.org/entity/Q943803,Q943803,Uno von Troil,https://kulturpersoner.uppsalakyrkogardar.se/u...,uno-von-troil
117,http://www.wikidata.org/entity/Q6043915,Q6043915,Viktor Persson,https://kulturpersoner.uppsalakyrkogardar.se/v...,viktor-persson
118,http://www.wikidata.org/entity/Q4983741,Q4983741,Vivi Laurent-Täckholm,https://kulturpersoner.uppsalakyrkogardar.se/v...,vivi-tackholm
119,http://www.wikidata.org/entity/Q6197251,Q6197251,Yngve Svalander,https://kulturpersoner.uppsalakyrkogardar.se/y...,yngve-svalander


compare href 

In [7]:
dfmerge = pd.merge(WDKulturGravar, Kulturgravar,how='outer', on='href',indicator=True)

In [8]:
dfmerge['_merge'] = dfmerge['_merge'].str.replace('left_only','Wikidata_only').str.replace('right_only','Kulturgravar_only')
dfmerge.rename(columns={"_merge": "Kulturgravar_Wikidata"},inplace = True)


In [9]:
dfmerge["Kulturgravar_Wikidata"].value_counts()

both    130
Name: Kulturgravar_Wikidata, dtype: int64

In [10]:
KulturgravarNotBoth = dfmerge[~(dfmerge["Kulturgravar_Wikidata"] == "both")].copy() 
KulturgravarNotBoth


,person,qid,personLabel,refURL,href,kulturgravname,Kulturgravar_Wikidata


In [11]:
KulturgravarNotBoth["refURL"]


Series([], Name: refURL, dtype: object)

In [12]:
Kulturgravaronly = dfmerge[dfmerge["Kulturgravar_Wikidata"] == "Kulturgravar_only"].copy() 

In [13]:
Kulturgravaronly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   person                 0 non-null      object
 1   qid                    0 non-null      object
 2   personLabel            0 non-null      object
 3   refURL                 0 non-null      object
 4   href                   0 non-null      object
 5   kulturgravname         0 non-null      object
 6   Kulturgravar_Wikidata  0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes


In [14]:
Kulturgravaronly["url"] = "https://kulturpersoner.uppsalakyrkogardar.se/" + Kulturgravaronly["href"] 
pd.set_option('column_space', 20)
pd.set_option('max_colwidth', 300)
Kulturgravaronly = Kulturgravaronly.reset_index(drop=True)  

Kulturgravaronly[["kulturgravname","url"]]

,kulturgravname,url


In [15]:
end = datetime.now()
print("Ended: ", end)
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2020-09-29 13:51:07.428835
Time elapsed (hh:mm:ss.ms) 0:00:01.895286
